In [1]:
import parser
import numpy as np
import gzip, msgpack
from torch.nn.modules import PairwiseDistance
import random
import torch 
import torch.nn as nn
from torch.autograd import Variable
import scipy.sparse as sp
import time

In [2]:
from sklearn.feature_extraction.text import HashingVectorizer
N_FEATURES = 2**18
hv = HashingVectorizer(n_features=N_FEATURES,non_negative=True, stop_words='english')
# try default n-gram 1-gram,2-gram,3-gram

In [3]:
class SmallReference:
    def __init__(self, identifiers):
        self.identifiers = identifiers

class SmallPaper:
    def __init__(self, title, identifiers, authors, abstract, refs):
        self.title = title
        self.authors = authors
        self.abstract = hv.transform([abstract])
        self.references = [SmallReference(ref.identifiers) for ref in refs]
        self.identifiers = identifiers

In [4]:
def get_doi(paper):
    _ref_dict = dict()
    for x in paper.identifiers:
#        @todo lowecase
        _ref_dict[x.key_type] = x.key.lower()
    if 'doi' in _ref_dict:
        return _ref_dict['doi']
    elif 'ieee' in _ref_dict:
        return _ref_dict['ieee']
    elif 'semanticscholar' in _ref_dict:
        return _ref_dict['semanticscholar']
    elif 'adsa' in _ref_dict:
        return _ref_dict['adsa']
    else:
        return False

    
def update_authors(ident_to_auth, authors_to_papers, paper):
    for author in paper.authors:
        author_name = author.name.name
        
        if not author_name:
            continue
        for identifier in author.identifiers:
            result = ident_to_auth.get(identifier.key.lower(), False) 
            if result:
                author_name = str(result)
                break
        for identifier in author.identifiers:
            ident_to_auth[identifier.key.lower()] = author_name
        if not authors_to_papers.get(author_name, False):
            authors_to_papers[author_name] = list()
        authors_to_papers[author_name].append(get_doi(paper))

In [ ]:


identies_set = dict()

_paper_dict = dict()
myiter = 0
count=0
_ref_dict = dict()
_training_set = list()
_validation_set = list()
_test_set = list()
_ident_to_auth_dict = dict()
_authors_to_papers_dict = dict()
with gzip.open("NN_Papers.msgpack.gz", "rb") as nn_papers_out:
    unpacker = msgpack.Unpacker(nn_papers_out, encoding='utf-8')
    for _paper in unpacker:
        count+=1
        paper = parser.Paper.deserialize(_paper)
        
        paper_doi = get_doi(paper)
        if not (paper_doi and paper.abstract):
            continue
        else:
            myiter+=1
        paper = SmallPaper(paper.title, paper.identifiers, paper.authors, paper.abstract, paper.references)
        if myiter>=200000:
            break
        #calculate statistics
        for iden in paper.identifiers:
            if identies_set.get(iden.key_type, False):
                identies_set[iden.key_type]+=1
            else:
                identies_set[iden.key_type]=1
        #end
        
        _ref_dict[paper_doi] = list()
 
        update_authors(_ident_to_auth_dict, _authors_to_papers_dict, paper)
        _paper_dict[paper_doi] = paper
        if myiter<=160000:
            _training_set.append(paper_doi)
        elif myiter<=180000: 
            _test_set.append(paper_doi)
        else:
            _validation_set.append(paper_doi)

print('total count: {0}'.format(count))
print(identies_set)

In [ ]:
def update_references(paper_dict, ref_dict):
    count = 0
    for paper_doi,paper in paper_dict.items():
        for ref in paper.references:
            ref_doi = get_doi(ref)
            if not ref_doi:
                continue
            result = ref_dict.get(ref_doi, False)
            if result!=False:
                count+=1
                ref_dict[paper_doi].append(ref_doi)
    print(count)
update_references(_paper_dict, _ref_dict)

In [ ]:
list(paper_list.keys())[6]
get_doi(_paper_dict['10.1109/tnnls.2015.2392563'].references[4])

In [ ]:
abstract_training_set = [paper.abstract for paper in training_set]
abstract_training_set_hashed = hv.transform(abstract_training_set)

In [ ]:
def make_var(abstract):
    dense = abstract.todense()
    torch_tensor = Variable(torch.from_numpy(dense).type(dtype))
    return torch_tensor

In [ ]:

random.randint(0,100)

In [ ]:
identies_set = dict()
for paper in paper_list.values():
    for iden in paper.identifiers:
        if identies_set.get(iden.key_type, False):
            identies_set[iden.key_type]+=1
        else:
            identies_set[iden.key_type]=1
print(identies_set)
len(paper_list)

In [ ]:



# def get_triple(val_set):
#     count = 0
#     t0 = 0
#     t1 =0 
#     for item in val_set:
#         count += 1
#         _ref_doi = get_doi(item)
#         if _ref_doi not in ref_dict.keys():
#             continue
#         _ref_list = ref_dict[_ref_doi]
        
#         for _ref in _ref_list:
#             if paper_list.get(_ref, False) in val_set:
#                 _neg_example = None
#                 while(not _neg_example):
#                     _neg_id = random.randint(0,len(val_set)-1)
#                     _neg_doi = get_doi(val_set[_neg_id])
#                     if (_neg_doi != _ref_doi) and (_neg_doi not in _ref_list):
#                         neg_ref_dict = ref_dict[_neg_doi]
#                         if not (_ref_doi in neg_ref_dict or _ref in neg_ref_dict):
#                             _neg_example = val_set[_neg_id]
# #                 t1 = time.time()
# #                 print('time to sample triple: {0}'.format(t1-t0))
#                 yield (item.abstract, paper_list[_ref].abstract, _neg_example.abstract)
# #                 t0 = time.time()

In [ ]:
BATCH_SIZE = 100

def prepare_triples(val_set):
    count = 0
    result_list = list()
    batch_list_1 = None
    batch_list_2 = None
    batch_list_3 = None
    print(len(val_set))
    for item in val_set:
        _ref_doi = get_doi(item)
        if _ref_doi not in ref_dict.keys():
            continue
        _ref_list = ref_dict[_ref_doi]
        if len(_ref_list) == 0:
            print(get_doi(item))
        for _ref in _ref_list:
            
            if paper_list.get(_ref, False) in val_set:
                _neg_example = None
                while(not _neg_example):
                    _neg_id = random.randint(0,len(val_set)-1)
                    _neg_doi = get_doi(val_set[_neg_id])
                    if (_neg_doi != _ref_doi) and (_neg_doi not in _ref_list):
                        neg_ref_dict = ref_dict[_neg_doi]
                        if not (_ref_doi in neg_ref_dict or _ref in neg_ref_dict):
                            _neg_example = val_set[_neg_id]
                            count+=1
                
                if batch_list_1 == None:
                    batch_list_1 = item.abstract
                    batch_list_2 = paper_list[_ref].abstract
                    batch_list_3 = _neg_example.abstract
                else:
                    batch_list_1 = sp.vstack((batch_list_1, item.abstract))
                    batch_list_2 = sp.vstack((batch_list_2, paper_list[_ref].abstract))
                    batch_list_3 = sp.vstack((batch_list_3, _neg_example.abstract))
                
                if batch_list_1.shape[0]==BATCH_SIZE:
                    result_list.append((batch_list_1, batch_list_2, batch_list_3))
                    batch_list_1 = None
    print(count)
    return result_list


    

In [ ]:
result_list = prepare_triples(training_set[:200])

In [ ]:
ref_dict['10.1117/12.24153']

In [ ]:



dtype = torch.FloatTensor
def get_batch(iterator, batch_size=BATCH_SIZE):
    a1 = None
    a2 = None
    a3 = None
    for x1,x2,x3 in iterator:
        if a1 == None:
            a1=x1
            a2=x2
            a3=x3
        else:
            a1 = sp.vstack((a1,x1))
            a2 = sp.vstack((a2,x2))
            a3 = sp.vstack((a3,x3))            
        if a1.shape[0]==batch_size:
            dense_a1 = a1.todense()
            dense_a2 = a2.todense()
            dense_a3 = a3.todense()
            tor_a1 = torch.from_numpy(dense_a1).type(dtype)
            tor_a2 = torch.from_numpy(dense_a2).type(dtype)
            tor_a3 = torch.from_numpy(dense_a3).type(dtype)
            yield (Variable(tor_a1),
                   Variable(tor_a2),
                   Variable(tor_a3))
            a1 = None
            a2 = None
            a3 = None


# input data sparse matrix


In [ ]:
tri_iter = get_triple(training_set)
x1,x2,x3 = next(tri_iter)
# x1 = sp.vstack((x1,x2),dtype=dtype)
# x1.todense()

batch_iterator = get_batch(tri_iter)
kk1,kk2,kk3 = next(batch_iterator)
kk1

In [ ]:
hv.transform(['avasd', 'asdfwe sdfasdfwe werwe adsfsd']).todense()

In [ ]:
class DistRes(nn.Module):

    def __init__(self):
        super(DistRes, self).__init__()
        self.layer1 = nn.Linear(N_FEATURES, 100)
        self.layer2 = nn.PairwiseDistance(2)

    def forward(self, x1,x2):
#         print('typex: {0}'.format(type(x1)))
#         print('typey: {0}'.format(type(x2)))
        res1 = self.layer1(x1)
        res2 = self.layer1(x2)
        # normalize vector L2-normalization
        return self.layer2(res1,res2)
    
# linear/RelU/linear
# dropout input layer
# help as they authored by same person

In [ ]:

dist_model = DistRes()

In [ ]:

# arg1 = Variable(torch.from_numpy(hv.transform([a1.abstract]).todense()))
# arg2 = Variable(torch.from_numpy(hv.transform([a2.abstract]).todense()))

# arg1 = Variable(torch.rand(2,5), requires_grad=True)
# arg2 = Variable(torch.rand(2,5), requires_grad=True)
# arg3 = Variable(torch.rand(2,5), requires_grad=True)



In [ ]:
#check the distance Loss, just to see the difference
# maybe pos and pos close

crit = nn.HingeEmbeddingLoss(1)
optim = torch.optim.SGD(dist_model.parameters(), lr=1e-3)
# local_dist(z)

In [ ]:
ones_var = Variable(torch.ones(BATCH_SIZE))
neg_var = Variable(torch.ones(BATCH_SIZE)*(-1))

def train_networks(model, criteria, optimizer, paper, p_pos, p_neg):
                    
    loss_pos = criteria.forward( model(paper, p_pos), ones_var)
    optimizer.zero_grad()
    gradCrit = loss_pos.backward()
    optimizer.step()

    loss_neg = criteria.forward( model(paper,p_neg), neg_var)
    optimizer.zero_grad()
    gradCrit = loss_neg.backward()
    optimizer.step()
    
    return model(paper, p_pos), model(paper,p_neg)




In [ ]:
import time


In [ ]:
epochs = 1
for i in range(epochs):
    aver_pos = list()
    aver_neg = list()
    triple_iterator = get_triple(training_set)
    batch_iterator = get_batch(triple_iterator)
    batch=0
    t0=0
    t1=0
    for arg1,arg2,arg3 in batch_iterator:
        t0 = time.time()
        print('sampling time: {0}'.format(t0-t1))
        ap, an = train_networks(model=dist_model, criteria=crit, optimizer=optim, paper=arg1, p_pos=arg2, p_neg=arg3)
        t1 = time.time()
        total= t1-t0
        aver_pos.append(ap.data.mean())
        aver_neg.append(an.data.mean())
        print('Epoch: {0}. Batch: {3}. Time {4} av_pos: {1}. av_neg: {2}'.format(i, np.mean(aver_pos), 
                                                                                 np.mean(aver_neg), 
                                                                                 batch, total))
        batch+=1
    print('Epoch: {0}. av_pos: {1}. av_neg: {2}'.format(i, np.mean(aver_pos), np.mean(aver_neg)))
    


In [ ]:
triple_iterator1 = get_triple(training_set)
batch_iterator1 = get_batch(triple_iterator1)
arg1,arg2,arg3 = next(batch_iterator1)
k = dist_model(arg1,arg2)

In [ ]:

# def calculate_distances(model,data_set):

#     stacked_matrix = data_set[0].abstract
#     for item in data_set[1:]:
#         stacked_matrix = sp.vstack((stacked_matrix, item.abstract))
        
#     res = model.layer1.forward(make_var(stacked_matrix)).data.numpy()
#     return res

# stacked_matrix = calculate_distances(dist_model, training_set)

In [ ]:
stacked_matrix.shape